# Studying the Small Scale Environment
## Relating Dwarf Galaxy Properties to Their Distance to Nearest Neighbor 

## Reading in NSA FITS file

### NSA (NASA Sloan Atlas)


 

In [1]:
#import astropy
from astropy.io import fits

from astropy.table import Table

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
#Reading in data file

fits_NSA_file = r'C:\Users\caeho\OneDrive\Documents\University of Rochester REU\NSA_v1_0_1_VAGC_vflag-V2-VF.fits'
#data = r'C:\Users\caeho\OneDrive\Documents\University of Rochester REU\NSA_v1_0_1_VAGC_vflag-V2-VF.fits'

hdul = fits.open(fits_NSA_file) 

In [3]:
hdul.info()

Filename: C:\Users\caeho\OneDrive\Documents\University of Rochester REU\NSA_v1_0_1_VAGC_vflag-V2-VF.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU    330   641409R x 141C   [19A, 27A, D, D, J, J, J, J, J, J, E, E, 7A, E, I, B, I, 3A, E, E, J, E, 7E, 7E, I, 7E, 7E, 7E, 7E, 7E, 5E, 7E, E, E, E, E, D, D, 7B, 105E, 105E, 105E, 105E, 105E, 105E, 7E, 7E, 7E, 7E, E, E, E, E, 7E, 7E, E, E, E, 7E, 7E, 7J, J, J, 8A, 15E, E, E, E, E, J, I, J, D, D, E, 6A, 27A, 8A, J, D, D, E, E, E, E, E, E, E, E, 7E, 7E, 7E, 7E, E, 7E, 7E, 7E, 7E, I, 7E, 7E, 7E, 7E, 5E, E, 7E, E, E, E, D, D, D, D, K, K, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, K, K]   


In [4]:
# Names of all the collumns in the data base

data_header = hdul[1].header
data_header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 4091 / length of dimension 1                          
NAXIS2  =               641409 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                  141 / number of table fields                         
TTYPE1  = 'IAUNAME '                                                            
TFORM1  = '19A     '                                                            
TTYPE2  = 'SUBDIR  '                                                            
TFORM2  = '27A     '                                                            
TTYPE3  = 'RA      '        

In [6]:
data = hdul[1].data

### Creating a smaller data table with only data I will need

In [7]:
data_table = Table()

data_table['aimc'] = data['aimc']
data_table['IN_DR7_LSS'] = data['IN_DR7_LSS']
data_table['RA'] = data['RA']
data_table['DEC'] = data['DEC']
data_table['Z'] = data['Z']
data_table['ELPETRO_ABSMAG_r'] = data['ELPETRO_ABSMAG'][:,4]
data_table['u_r'] = data['u_r']
data_table['sSFR'] = data['sSFR']
data_table['HImass'] = data['HImass']
data_table['flag3727'] = data['flag3727']
data_table['t3'] = data['t3']
data_table['Z12logOH'] = data['Z12logOH']
data_table['Zerr'] = data['Zerr']
data_table['logNO'] = data['logNO']
data_table['NOerr'] = data['NOerr']
data_table['vflag_VF'] = data['vFlag_VF']
data_table['ID#'] = [i for i in range(len(data_table))]
data_table['BPTclass'] = data['BPTclass']
data_table['ELPETRO_MASS'] = data['ELPETRO_MASS'] #using for the change in sSFR calcuation for metallicity

'''
Here is what will be added for the metallicity calculations

data_table['N2'] = data['N2']
data_table['O3'] = data['O3']
data_table['O2'] = data['O2']
data_table['Ha'] = data['Ha']
data_table['Hb'] = data['Hb']

Or these could've already been precalculated:

data_table['N2'] = data['N2'] (If this is not precalcuated will name it N2_H once calculated from N2)
data_table['O3N2'] = data['O3N2']
data_table['N2O2'] = data['N2O2']


'''

"\nHere is what will be added for the metallicity calculations\n\ndata_table['N2'] = data['N2']\ndata_table['O3'] = data['O3']\ndata_table['O2'] = data['O2']\ndata_table['Ha'] = data['Ha']\ndata_table['Hb'] = data['Hb']\n\nOr these could've already been precalculated:\n\ndata_table['N2'] = data['N2'] (If this is not precalcuated will name it N2_H once calculated from N2)\ndata_table['O3N2'] = data['O3N2']\ndata_table['N2O2'] = data['N2O2']\n\n\n"

In [ ]:
mask = data_table['Z']>0     #removing objects that are not galaxies
data_table = data_table[mask]

In [ ]:
mask = data_table['ELPETRO_ABSMAG_r']>-25 #Removing outlier galaxies with M < -25
data_table = data_table[mask]

In [ ]:
data_table[:10] # viewing first 5 rows of data as reference

# Calcuating New Metallicities (N2 Method, O3N2 Method, and N2O2 Method)

### This is an outline attempt of what I will code once I have access to the data I need to start)

In [ ]:
data_table['N2_H'] = np.nan
data_table['O3N2'] = np.nan
data_table['N2O2'] = np.nan
data_table['log_M_sSFR'] = np.nan

for i in range(len(data_table)):
    
    #I might not need to put these in a for loop
    
    data_table['N2_H'][i] = data_table['N2'][i] / data_table['Ha'][i]
    data_table['O3N2'][i] = data_table['O3'][i] / data_table['Hb'][i] / data_table['N2'][i]
    data_table['N2O2'][i] = data_table['N2'][i] / data_table['O3'][i]
    
    data_table['log_M_sSFR'][i] = 283.728 - 116.265 * np.log(data_table['ELPETRO_MASS'][i]) + 17.4403 * 
    (np.log(data_table['ELPETRO_MASS'][i]))**2 - 1.17146 * (np.log(data_table['ELPETRO_MASS'][i]))**3 + 0.0296526 *
    (np.log(data_table['ELPETRO_MASS'][i]))**4
    
    
    #log(SSFR)M⋆ =283.728−116.265×logM⋆+ 17.4403×logM⋆ 2−1.17146×logM⋆ 3+0.0296526×logM⋆ 4.
    

In [ ]:
data_table['change_log_sSFR'] = np.nan

for i in range(len(data_table)):
    data_table['change_log_sSFR'][i] = np.log(data_table['sSFR'][i]) - data_table['log_M_sSFR'][i]

In [ ]:
data_table['Z12logOH_N2'] = np.nan
data_table['Z12logOH_O3N2'] = np.nan
data_table['Z12logOH_N2O2'] = np.nan

for i in range(len(data_table)):
    
    data_table['Z12logOH_N2'] = 9.12 + 0.58 * np.log(data_table['N2_H'][i]) - 0.19 * data_table['change_log_sSFR'][i]
    
    data_table['Z12logOH_O3N2'] = 8.98 - 0.32 * np.log(data_table['O3N2'][i]) - 0.18 * data_table['change_log_sSFR'][i]
    
    data_table['Z12logOH_N2O2'] = 9.20 + 0.54 * np.log(data_table['N2O2'][i]) - 0.36 * data_table['change_log_sSFR'][i]
    
    
    #12+log(O/H)N2=9.12+0.58×log(N2) −0.19×∆log(SSFR)
    #12+log(O/H)O3N2=8.98−0.32×log(O3N2) −0.18×∆log(SSFR)
    #12+log(O/H)N2O2=9.20+0.54×log(N2O2) −0.36×∆log(SSFR).

## Galatic Coordinate Plot

using this a guide https://astronomy.stackexchange.com/questions/32601/how-do-i-plot-galactic-coordinates-using-matplotlib-and-astropy-in-python

In [ ]:
import astropy.units as u
import astropy.coordinates as coord
import astropy.units as u
from astropy.io import ascii
from astropy.coordinates import SkyCoord

plt.rcParams["figure.figsize"] = (10,6) #increase size of figure

eq = SkyCoord(180-data_table['RA'], data_table['DEC'], unit=u.deg)
gal = eq.galactic

gal = SkyCoord(180-data_table['RA'], data_table['DEC'], frame='galactic', unit=u.deg)


plt.subplot(111, projection='aitoff')
plt.grid(True)
plt.scatter(gal.l.wrap_at('180d').radian,
            gal.b.radian, marker='o', c='k', s=1, lw=0); #added same marker appearance as above code


# Finding Nearest Neighbor & Distance

### Table of only Dwarf Galaxies (ABSMAG_r >-17)

reference https://stackoverflow.com/questions/60448562/remove-row-from-astropy-table

In [ ]:
import pandas as pd
import numpy as np

dwarf_table = data_table
mask = (dwarf_table['ELPETRO_ABSMAG_r']<-17) | (dwarf_table['IN_DR7_LSS']!= 1) #removing galaxies w/ magnitude larger than -17
dwarf_table = dwarf_table[~mask]
RA_boolean = (dwarf_table['RA']>110) & (dwarf_table['RA']<270) #removing strips and area outside main area
dwarf_table = dwarf_table[RA_boolean]
dwarf_table[:5]



### Galactic plot of Dwarf Galaxies
            

In [ ]:
plt.rcParams["figure.figsize"] = (10,6) #increase size of figure

eq = SkyCoord(180-dwarf_table['RA'], dwarf_table['DEC'], unit=u.deg) #centering galaxies
gal = eq.galactic

gal = SkyCoord(180-dwarf_table['RA'], dwarf_table['DEC'], frame='galactic', unit=u.deg)


plt.subplot(111, projection='aitoff')
plt.grid(True)

plt.scatter(gal.l.wrap_at('180d').radian,
            gal.b.radian, marker='o', c='k', s=1, lw=0); 


### Distance Method #1: 2D Distance 

Calculating the relative velocity b/t each galaxy

Equation to calculate relative velocity: $v_{rel} = |z_{a} - z_{b}|c$

All neighbors galaxies have a maximum relative velocity: $v_{rel} < 300  km/s$

info on SkyCoord under the section "Distance" https://docs.astropy.org/en/stable/coordinates/index.html

In [ ]:
from astropy import constants as const
from astropy import units as u
from astropy.coordinates import SkyCoord

h=1
H0 = 100*h*u.km/u.s/u.Mpc #Hubble's Constant

#creating the new columns
dwarf_table['2D_neighbor_ID'] = -1
dwarf_table['2D_neighbor_dist'] = np.nan

#columns for virial radius normalization 
dwarf_table['2D_neighbor_ID_vir'] = -1
dwarf_table['2D_neighbor_dist_vir'] = np.nan

#gamma ratio in virial radius calculation
data_table['gamma_ratio'] = 1 
data_table['gamma_ratio'][data_table['aimc'] == 1] = 2 
data_table['gamma_ratio'][data_table['aimc'] == 2] = 2
                          


for i in range(len(dwarf_table)):
    neighbor_list=(abs(dwarf_table['Z'][i]-data_table['Z'])*const.c) < 300*u.km/u.s #galaxies in relative velocity constraint
    
    di = data_table['ID#'] == dwarf_table['ID#'][i] #finding the dwarf galaxy inside larger table
    neighbor_list[di]= False #removing from its neighbor list
   
    #avg distance to systems (r average)
    avg_dist = .5*(const.c.to('km/s')*(dwarf_table['Z'][i]+data_table['Z'][neighbor_list])/H0) 
    
    #galaxy positions to calcuate their separation
    dwarf_pos = SkyCoord(ra=dwarf_table['RA'][i]*u.degree, dec=dwarf_table['DEC'][i]*u.degree, distance=avg_dist)
    neighbor_pos = SkyCoord(ra=data_table['RA'][neighbor_list]*u.degree, 
                            dec=data_table['DEC'][neighbor_list]*u.degree, distance=avg_dist)
    
    neighbor_dist=neighbor_pos.separation_3d(dwarf_pos) #separation calculation of all possible neighbors
    
    nearby_neigh_index = np.argmin(neighbor_dist) #finding closest neighbor
    
    dwarf_table['2D_neighbor_dist'][i] = neighbor_dist[nearby_neigh_index].value #unit Mpc, adding to dwarf table
    
    neighbor_id = data_table['ID#'][neighbor_list] 
    dwarf_table['2D_neighbor_ID'][i] = neighbor_id[nearby_neigh_index] #adding neighbor galaxy's ID to dwarf table
    
    
    # Doing calculation with virial radius normalization
    
    M = data_table['ELPETRO_ABSMAG_r'][neighbor_list] #absolute magnitude of neighboring galaxies
    
    gamma_ratio = data_table['gamma_ratio'][neighbor_list] #gamma ratio of neighboring galaxies
    
    
    L_ratio = 10**((-20-M)/2.5) #luminosity ratio for virial radius calcuation
    
    r_vir = ((3*gamma_ratio*L_ratio)/(4*740*0.0223*np.pi))**(1./3) # calculating virial radius of all neighboring galaxies
    
    
    neighbor_dist_vir = neighbor_dist / r_vir # normalizing distance
    
    nearby_neigh_index_vir = np.nanargmin(neighbor_dist_vir) #finding closest neighbor with normalized virial distance
    
    dwarf_table['2D_neighbor_dist_vir'][i] = neighbor_dist_vir[nearby_neigh_index_vir].value #unit , adding to dwarf table
    
    
    dwarf_table['2D_neighbor_ID_vir'][i] = neighbor_id[nearby_neigh_index_vir] #adding neighbor galaxy's ID to dwarf table
    
    
    

In [ ]:
dwarf_table[:10]

### Distance Method #2: 3D Distance

In [ ]:

#creating the new columns
dwarf_table['3D_neighbor_ID'] = -1
dwarf_table['3D_neighbor_dist'] = np.nan

#columns for virial radius normalization 
dwarf_table['3D_neighbor_ID_vir'] = -1
dwarf_table['3D_neighbor_dist_vir'] = np.nan

h=1
H0 = 100*h*u.km/u.s/u.Mpc #Hubble's Constant


#gamma ratio in virial radius calculation
data_table['gamma_ratio'] = 1 
data_table['gamma_ratio'][data_table['aimc'] == 1] = 2 
data_table['gamma_ratio'][data_table['aimc'] == 2] = 2



for i in range(len(dwarf_table)):
    di = ~(data_table['ID#'] == dwarf_table['ID#'][i]) #finding the dwarf galaxy inside larger table
    #data_table[di]= False #removing dwarf from the full table to not count itself
    
    #distances to systems
    dwarf_r = const.c.to('km/s')*dwarf_table['Z'][i]/H0 
    gal_r = const.c.to('km/s')*data_table['Z'][di]/H0

    
    #galaxy positions to calcuate their separation
    dwarf_pos = SkyCoord(ra=dwarf_table['RA'][i]*u.degree, dec=dwarf_table['DEC'][i]*u.degree, distance=dwarf_r)
    gal_pos = SkyCoord(ra=data_table['RA'][di]*u.degree, 
                            dec=data_table['DEC'][di]*u.degree, distance=gal_r)
    neighbor_dist=gal_pos.separation_3d(dwarf_pos)
    
    nearby_neigh_index = np.argmin(neighbor_dist) #finding the nearest neighbor
    
    
    dwarf_table['3D_neighbor_dist'][i] = neighbor_dist[nearby_neigh_index].value #unit Mpc
     
    neighbor_id = data_table['ID#'][di] 
    dwarf_table['3D_neighbor_ID'][i] = neighbor_id[nearby_neigh_index] #ID# of the closest neighbor galaxy
    
    # Doing calculation with virial radius normalization
    
    M = data_table['ELPETRO_ABSMAG_r'][di] #[data_table] #absolute magnitude of neighboring galaxies
    
    gamma_ratio = data_table['gamma_ratio'][di] #[data_table] #gamma ratio of neighboring galaxies
    
    
    L_ratio = 10**((-20-M)/2.5) #luminosity ratio for virial radius calcuation
    
    r_vir = ((3*gamma_ratio*L_ratio)/(4*740*0.0223*np.pi))**(1./3) # calculating virial radius of all neighboring galaxies
    
    
    neighbor_dist_vir = neighbor_dist / r_vir # normalizing distance
    

    
    nearby_neigh_index_vir = np.nanargmin(neighbor_dist_vir) #finding closest neighbor with normalized virial distance
    
    dwarf_table['3D_neighbor_dist_vir'][i] = neighbor_dist_vir[nearby_neigh_index_vir].value #unit , adding to dwarf table
    
    
    neighbor_id = data_table['ID#'][di]
    dwarf_table['3D_neighbor_ID_vir'][i] = neighbor_id[nearby_neigh_index_vir] #adding neighbor galaxy's ID to dwarf table
    
    
    
    

In [ ]:
dwarf_table[:5]

In [ ]:
dwarf_wall = dwarf_table['vflag_VF']==0
dwarf_void = dwarf_table['vflag_VF']==1
dwarf_other = ~(dwarf_wall | dwarf_void)

plt.figure(tight_layout = True)
plt.plot(dwarf_table['2D_neighbor_dist'][dwarf_wall], dwarf_table['u_r'][dwarf_wall], "k^", alpha = 0.5, label = 'wall')
plt.plot(dwarf_table['2D_neighbor_dist'][dwarf_void], dwarf_table['u_r'][dwarf_void], "ro", alpha = 0.2, label = 'void')
plt.scatter(dwarf_table['2D_neighbor_dist'][dwarf_other], dwarf_table['u_r'][dwarf_other], facecolors='none', edgecolors='g', alpha = 0.5, label = 'other')
plt.legend()
plt.xlabel('Distance to Nearest Neighbor [$h^{-1}Mpc]$')
plt.ylabel('u-r');


In [ ]:
dwarf_table['ID#'].name = 'ID'

In [ ]:
#dwarf_table.write('dwarf_table.txt', format = 'ascii.commented_header', overwrite = True)

In [ ]:
#data_table.write('gal_table.txt', format = 'ascii.commented_header', overwrite = True)

In [ ]:
#hdul.close() #closes file